In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Setup and Imports
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import os
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from tensorflow.keras.layers import Bidirectional, GRU
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")

✓ All imports successful


In [3]:
def load_and_filter_fold(i):
    train_dir = f'/content/drive/MyDrive/split_data/fold{i}/train.csv'
    test_dir = f'/content/drive/MyDrive/split_data/fold{i}/test.csv'
    train_df = pd.read_csv(train_dir)
    test_df = pd.read_csv(test_dir)

    train_labels = list(train_df['room'].unique())
    test_labels = list(test_df['room'].unique())
    common_labels = list(set(train_labels) & set(test_labels))

    train_df = train_df[train_df['room'].isin(common_labels)].reset_index(drop=True)
    test_df = test_df[test_df['room'].isin(common_labels)].reset_index(drop=True)

    return train_df, test_df

# Load all 4 folds
train_df_1, test_df_1 = load_and_filter_fold(1)
train_df_2, test_df_2 = load_and_filter_fold(2)
train_df_3, test_df_3 = load_and_filter_fold(3)
train_df_4, test_df_4 = load_and_filter_fold(4)

print("✓ All folds loaded")

✓ All folds loaded


In [4]:
def set_seeds(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

def create_room_groups(df):
    df = df.sort_values('timestamp').reset_index(drop=True)
    df['room_group'] = (df['room'] != df['room'].shift()).cumsum()
    return df

def create_beacon_count_vectors(df):
    """Aggregates readings into 1s vectors. Handles data with or without 'room_group'."""
    vectors = []
    has_groups = 'room_group' in df.columns # Check if we are in 'training' mode

    for _, group in df.groupby('timestamp'):
        beacon_counts = group['mac address'].value_counts()
        total_readings = len(group)

        vector = [0.0] * 23
        for beacon_id, count in beacon_counts.items():
            if 1 <= beacon_id <= 23:
                vector[int(beacon_id) - 1] = count / total_readings

        entry = {
            'timestamp': group['timestamp'].iloc[0],
            'room': group['room'].iloc[0],
            'beacon_vector': vector
        }

        if has_groups:
            entry['room_group'] = group['room_group'].iloc[0]

        vectors.append(entry)

    return pd.DataFrame(vectors)

def create_sequences_from_groups(vector_df, min_length=3, max_length=50):
    """Used for Training: Creates clean sequences where the room is constant."""
    sequences = []
    labels = []

    for (room, room_group), group in vector_df.groupby(['room', 'room_group']):
        group = group.sort_values('timestamp').reset_index(drop=False)
        seq_length = len(group)

        if seq_length < min_length:
            continue

        if seq_length > max_length:
            group = group.tail(max_length)

        sequence = [row['beacon_vector'] for _, row in group.iterrows()]
        sequences.append(sequence)
        labels.append(room)

    return sequences, labels

def create_sliding_windows_by_day(vector_df, window_size=10):
    """Used for Inference: Creates a sequence for every frame, respecting day boundaries."""
    sequences = []
    labels = []

    # Ensure chronological order and group by day
    vector_df['dt'] = pd.to_datetime(vector_df['timestamp'])
    vector_df['date'] = vector_df['dt'].dt.date

    for _, day_group in vector_df.groupby('date'):
        day_group = day_group.sort_values('timestamp').reset_index(drop=True)

        if len(day_group) >= window_size:
            vectors = list(day_group['beacon_vector'])
            rooms = list(day_group['room'])

            for i in range(len(vectors) - window_size + 1):
                window = vectors[i : i + window_size]
                sequences.append(window)
                # Goal: Predict the room at the final timestamp of the window
                labels.append(rooms[i + window_size - 1])

    return sequences, labels

def build_bidirectional_gru_model(input_shape, num_classes):
    """
    Bidirectional GRU Architecture
    """
    model = Sequential([
        Masking(mask_value=0.0, input_shape=input_shape),

        Bidirectional(GRU(128, return_sequences=True)),
        Dropout(0.3),

        Bidirectional(GRU(64, return_sequences=False)),
        Dropout(0.3),

        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

def create_bidirectional_windows_by_day(vector_df, window_size=10):
    """
    Creates 2 types of windows for each timestamp:
    1. Backward-looking: [t-9, t-8, ..., t-1, t]
    2. Forward-looking: [t, t+1, ..., t+8, t+9]

    Returns sequences for both window types + ground truth labels.
    Uses right-padding for cuDNN compatibility.
    """
    backward_sequences = []
    forward_sequences = []
    labels = []

    # Ensure chronological order and group by day
    vector_df['dt'] = pd.to_datetime(vector_df['timestamp'])
    vector_df['date'] = vector_df['dt'].dt.date

    for _, day_group in vector_df.groupby('date'):
        day_group = day_group.sort_values('timestamp').reset_index(drop=True)
        vectors = list(day_group['beacon_vector'])
        rooms = list(day_group['room'])
        day_length = len(vectors)

        if day_length < window_size:
            continue

        zero_vector = [0.0] * len(vectors[0])

        # For each position in the day
        for i in range(day_length):

            # === BACKWARD WINDOW: [t-9, ..., t] ===
            if i >= window_size - 1:
                # Full backward window available
                backward_window = vectors[i - window_size + 1 : i + 1]
            else:
                # Not enough history - take what we have and RIGHT-PAD
                backward_window = vectors[0 : i + 1]
                padding_needed = window_size - len(backward_window)
                backward_window = backward_window + [zero_vector] * padding_needed

            backward_sequences.append(backward_window)

            # === FORWARD WINDOW: [t, t+1, ..., t+9] ===
            if i + window_size <= day_length:
                # Full forward window available
                forward_window = vectors[i : i + window_size]
            else:
                # Not enough future - take what we have and RIGHT-PAD
                forward_window = vectors[i : day_length]
                padding_needed = window_size - len(forward_window)
                forward_window = forward_window + [zero_vector] * padding_needed

            forward_sequences.append(forward_window)

            # Ground truth label for this position
            labels.append(rooms[i])

    return backward_sequences, forward_sequences, labels

def ensemble_predictions_with_confidence(model, X_backward, X_forward):
    """
    Gets predictions from both window types and selects the one with highest confidence.
    Uses only backward and forward windows for stability.
    """
    # Get probability predictions from both models
    probs_backward = model.predict(X_backward, verbose=0)
    probs_forward = model.predict(X_forward, verbose=0)

    # Extract max confidence and predicted class for each
    conf_backward = np.max(probs_backward, axis=1)
    pred_backward = np.argmax(probs_backward, axis=1)

    conf_forward = np.max(probs_forward, axis=1)
    pred_forward = np.argmax(probs_forward, axis=1)

    # For each sample, choose prediction with highest confidence
    final_predictions = []

    for i in range(len(pred_backward)):
        # Compare confidences from both windows
        if conf_backward[i] > conf_forward[i]:
            final_predictions.append(pred_backward[i])
        else:
            final_predictions.append(pred_forward[i])

    return np.array(final_predictions)

def analyze_confidence_differences(model, X_backward, X_forward, sample_size=1000):
    """Debug: Analyze how different backward vs forward confidences are"""
    probs_backward = model.predict(X_backward[:sample_size], verbose=0)
    probs_forward = model.predict(X_forward[:sample_size], verbose=0)

    conf_backward = np.max(probs_backward, axis=1)
    conf_forward = np.max(probs_forward, axis=1)

    pred_backward = np.argmax(probs_backward, axis=1)
    pred_forward = np.argmax(probs_forward, axis=1)

    # Cases where predictions differ
    disagreements = pred_backward != pred_forward

    print(f"\n🔍 CONFIDENCE ANALYSIS (first {sample_size} samples):")
    print(f"  Predictions disagree: {disagreements.sum()} / {sample_size} ({100*disagreements.sum()/sample_size:.1f}%)")
    print(f"  Avg confidence (backward): {conf_backward.mean():.3f}")
    print(f"  Avg confidence (forward):  {conf_forward.mean():.3f}")

    if disagreements.sum() > 0:
        print(f"\n  When predictions DISAGREE:")
        print(f"    Avg conf difference: {np.abs(conf_backward[disagreements] - conf_forward[disagreements]).mean():.3f}")
        print(f"    Max conf difference: {np.abs(conf_backward[disagreements] - conf_forward[disagreements]).max():.3f}")
        print(f"    Backward higher: {(conf_backward[disagreements] > conf_forward[disagreements]).sum()}")
        print(f"    Forward higher:  {(conf_forward[disagreements] > conf_backward[disagreements]).sum()}")

print("✅ Bidirectional ensemble functions defined (Backward + Forward only)")

✅ Bidirectional ensemble functions defined (Backward + Forward only)


In [5]:
def run_pipeline_single_seed(train_df, test_df, seed, verbose=False, debug_confidence=False):
    """
    Run realistic pipeline with Bidirectional Ensemble (Backward + Forward) + Temporal Voting.
    Includes optional confidence analysis for debugging.

    1. Resets memory/seeds.
    2. Trains on pure segments.
    3. Infers with 2 sliding window directions (backward, forward).
    4. Ensemble: Chooses prediction with highest confidence.
    5. Smooths predictions with a Majority Vote filter.
    6. (Optional) Analyzes confidence distributions for debugging.
    """
    # 0. PREVENT MEMORY LEAKS & ENSURE SEEDING
    tf.keras.backend.clear_session()
    set_seeds(seed)

    # HYPERPARAMETERS
    window_size = 10     # Window size for both directions
    vote_window = 5      # The smoothing neighborhood (5 seconds)
    max_seq_length = 50

    # 1. Preprocessing
    train_df = create_room_groups(train_df)
    train_vectors = create_beacon_count_vectors(train_df)
    test_vectors = create_beacon_count_vectors(test_df)

    # 2. Sequence Creation
    X_train, y_train = create_sequences_from_groups(train_vectors, max_length=max_seq_length)

    # Create bidirectional windows for test
    X_test_backward, X_test_forward, y_test = \
        create_bidirectional_windows_by_day(test_vectors, window_size=window_size)

    # 3. Encoding & Padding
    label_encoder = LabelEncoder()
    label_encoder.fit(list(y_train) + list(y_test))
    y_train_encoded = label_encoder.transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    X_train_padded = pad_sequences(X_train, maxlen=max_seq_length, dtype='float32', padding='post', value=0.0)

    # Pad both test window types
    X_test_backward_padded = pad_sequences(X_test_backward, maxlen=max_seq_length, dtype='float32', padding='post', value=0.0)
    X_test_forward_padded = pad_sequences(X_test_forward, maxlen=max_seq_length, dtype='float32', padding='post', value=0.0)

    # 4. Train Model with Macro F1 Optimization
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train_encoded), y=y_train_encoded)
    class_weight_dict = dict(enumerate(class_weights))

    model = build_bidirectional_gru_model(input_shape=(max_seq_length, 23), num_classes=len(label_encoder.classes_))

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=0)
    ]

    # Use backward window for validation (standard approach)
    model.fit(
        X_train_padded, y_train_encoded,
        validation_data=(X_test_backward_padded, y_test_encoded),
        epochs=100, batch_size=32,
        class_weight=class_weight_dict,
        callbacks=callbacks, verbose=0
    )

    # 5. CONFIDENCE ANALYSIS (Optional Debug)
    if debug_confidence:
        print(f"\n{'='*80}")
        print(f"🔍 CONFIDENCE ANALYSIS - Seed {seed}")
        print(f"{'='*80}")

        # Analyze on full test set
        sample_size = min(1000, len(X_test_backward_padded))

        probs_backward = model.predict(X_test_backward_padded[:sample_size], verbose=0)
        probs_forward = model.predict(X_test_forward_padded[:sample_size], verbose=0)

        conf_backward = np.max(probs_backward, axis=1)
        conf_forward = np.max(probs_forward, axis=1)

        pred_backward = np.argmax(probs_backward, axis=1)
        pred_forward = np.argmax(probs_forward, axis=1)

        # Cases where predictions differ
        disagreements = pred_backward != pred_forward
        agreements = pred_backward == pred_forward

        print(f"\n📊 Overall Statistics (first {sample_size} samples):")
        print(f"  Total samples analyzed: {sample_size}")
        print(f"  Predictions AGREE:     {agreements.sum():4d} ({100*agreements.sum()/sample_size:5.1f}%)")
        print(f"  Predictions DISAGREE:  {disagreements.sum():4d} ({100*disagreements.sum()/sample_size:5.1f}%)")

        print(f"\n📈 Confidence Scores:")
        print(f"  Backward window - Mean: {conf_backward.mean():.4f}, Std: {conf_backward.std():.4f}")
        print(f"  Forward window  - Mean: {conf_forward.mean():.4f}, Std: {conf_forward.std():.4f}")

        if disagreements.sum() > 0:
            conf_diff_when_disagree = np.abs(conf_backward[disagreements] - conf_forward[disagreements])

            print(f"\n⚡ When Predictions DISAGREE ({disagreements.sum()} cases):")
            print(f"  Confidence difference:")
            print(f"    Mean:   {conf_diff_when_disagree.mean():.4f}")
            print(f"    Median: {np.median(conf_diff_when_disagree):.4f}")
            print(f"    Std:    {conf_diff_when_disagree.std():.4f}")
            print(f"    Min:    {conf_diff_when_disagree.min():.4f}")
            print(f"    Max:    {conf_diff_when_disagree.max():.4f}")

            # Distribution of differences
            print(f"\n  Confidence difference distribution:")
            print(f"    < 0.05 (very small): {(conf_diff_when_disagree < 0.05).sum():3d} ({100*(conf_diff_when_disagree < 0.05).sum()/disagreements.sum():5.1f}%)")
            print(f"    0.05-0.10:           {((conf_diff_when_disagree >= 0.05) & (conf_diff_when_disagree < 0.10)).sum():3d} ({100*((conf_diff_when_disagree >= 0.05) & (conf_diff_when_disagree < 0.10)).sum()/disagreements.sum():5.1f}%)")
            print(f"    0.10-0.20:           {((conf_diff_when_disagree >= 0.10) & (conf_diff_when_disagree < 0.20)).sum():3d} ({100*((conf_diff_when_disagree >= 0.10) & (conf_diff_when_disagree < 0.20)).sum()/disagreements.sum():5.1f}%)")
            print(f"    >= 0.20 (large):     {(conf_diff_when_disagree >= 0.20).sum():3d} ({100*(conf_diff_when_disagree >= 0.20).sum()/disagreements.sum():5.1f}%)")

            print(f"\n  Which window is more confident when they disagree:")
            backward_higher = (conf_backward[disagreements] > conf_forward[disagreements]).sum()
            forward_higher = (conf_forward[disagreements] > conf_backward[disagreements]).sum()
            print(f"    Backward higher: {backward_higher:3d} ({100*backward_higher/disagreements.sum():5.1f}%)")
            print(f"    Forward higher:  {forward_higher:3d} ({100*forward_higher/disagreements.sum():5.1f}%)")

            # Check correctness when they disagree
            y_test_sample = y_test_encoded[:sample_size]
            backward_correct = pred_backward[disagreements] == y_test_sample[disagreements]
            forward_correct = pred_forward[disagreements] == y_test_sample[disagreements]

            print(f"\n  Accuracy when predictions disagree:")
            print(f"    Backward correct: {backward_correct.sum():3d} / {disagreements.sum()} ({100*backward_correct.sum()/disagreements.sum():5.1f}%)")
            print(f"    Forward correct:  {forward_correct.sum():3d} / {disagreements.sum()} ({100*forward_correct.sum()/disagreements.sum():5.1f}%)")
            print(f"    Both wrong:       {(~backward_correct & ~forward_correct).sum():3d} / {disagreements.sum()} ({100*(~backward_correct & ~forward_correct).sum()/disagreements.sum():5.1f}%)")

        print(f"\n{'='*80}\n")

    # 6. BIDIRECTIONAL ENSEMBLE INFERENCE
    y_pred_raw_encoded = ensemble_predictions_with_confidence(
        model,
        X_test_backward_padded,
        X_test_forward_padded
    )

    # 7. TEMPORAL VOTING (Smoothing)
    def apply_temporal_voting(preds, v_window):
        """Applies a majority vote filter to smooth room predictions."""
        smoothed = []
        for i in range(len(preds)):
            start = max(0, i - v_window // 2)
            end = min(len(preds), i + v_window // 2 + 1)
            neighborhood = preds[start:end]
            smoothed.append(np.bincount(neighborhood).argmax())
        return np.array(smoothed)

    y_pred_voted_encoded = apply_temporal_voting(y_pred_raw_encoded, vote_window)
    y_pred = label_encoder.inverse_transform(y_pred_voted_encoded)

    # 8. Final Evaluation
    macro_f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    per_class_f1 = f1_score(y_test, y_pred, average=None, labels=label_encoder.classes_, zero_division=0)

    return {
        'seed': seed,
        'macro_f1': macro_f1,
        'per_class_f1': {label: f1 for label, f1 in zip(label_encoder.classes_, per_class_f1)}
    }

print("✓ Pipeline updated with confidence analysis (use debug_confidence=True to enable)")

✓ Pipeline updated with confidence analysis (use debug_confidence=True to enable)


In [6]:
# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
# Run 10 seeds for each of 4 folds
seeds = [42, 123, 456, 789, 2024, 3141, 5926, 8888, 1337, 9999]
folds = {
    1: (train_df_1, test_df_1),
    2: (train_df_2, test_df_2),
    3: (train_df_3, test_df_3),
    4: (train_df_4, test_df_4)
}

all_fold_results = {}

for fold_num, (train_df, test_df) in folds.items():
    print(f"\n{'='*80}")
    print(f"PROCESSING FOLD {fold_num}")
    print(f"{'='*80}\n")

    fold_results = []

    for idx, seed in enumerate(seeds):
        # Enable debug only for first seed of each fold
        debug_mode = (idx == 0)

        if debug_mode:
            print(f"  Running seed {seed} [WITH DEBUG]...")
        else:
            print(f"  Running seed {seed}...", end=" ")

        result = run_pipeline_single_seed(train_df, test_df, seed, verbose=False, debug_confidence=debug_mode)
        fold_results.append(result)
        print(f"Macro F1: {result['macro_f1']:.4f}")

    all_fold_results[fold_num] = fold_results

    # Calculate fold statistics
    macro_f1_scores = [r['macro_f1'] for r in fold_results]
    print(f"\n  Fold {fold_num} Summary:")
    print(f"    Mean Macro F1: {np.mean(macro_f1_scores):.4f} ± {np.std(macro_f1_scores):.4f}")
    print(f"    Min: {np.min(macro_f1_scores):.4f}, Max: {np.max(macro_f1_scores):.4f}")

print("\n" + "="*80)
print("ALL FOLDS COMPLETED!")
print("="*80)


PROCESSING FOLD 1

  Running seed 42 [WITH DEBUG]...

🔍 CONFIDENCE ANALYSIS - Seed 42

📊 Overall Statistics (first 1000 samples):
  Total samples analyzed: 1000
  Predictions AGREE:      492 ( 49.2%)
  Predictions DISAGREE:   508 ( 50.8%)

📈 Confidence Scores:
  Backward window - Mean: 0.3976, Std: 0.1277
  Forward window  - Mean: 0.3974, Std: 0.1270

⚡ When Predictions DISAGREE (508 cases):
  Confidence difference:
    Mean:   0.1090
    Median: 0.0829
    Std:    0.0941
    Min:    0.0009
    Max:    0.4135

  Confidence difference distribution:
    < 0.05 (very small): 172 ( 33.9%)
    0.05-0.10:           122 ( 24.0%)
    0.10-0.20:           124 ( 24.4%)
    >= 0.20 (large):      90 ( 17.7%)

  Which window is more confident when they disagree:
    Backward higher: 252 ( 49.6%)
    Forward higher:  256 ( 50.4%)

  Accuracy when predictions disagree:
    Backward correct: 154 / 508 ( 30.3%)
    Forward correct:  120 / 508 ( 23.6%)
    Both wrong:       234 / 508 ( 46.1%)


Macro F

KeyboardInterrupt: 

In [ ]:
# Save results to text file
with open('4fold_10seed_results.txt', 'w') as f:
    f.write("="*80 + "\n")
    f.write("4-FOLD CROSS-VALIDATION WITH 10 SEEDS PER FOLD\n")
    f.write("="*80 + "\n\n")

    # Overall summary
    all_macro_f1 = []
    for fold_num in [1, 2, 3, 4]:
        fold_scores = [r['macro_f1'] for r in all_fold_results[fold_num]]
        all_macro_f1.extend(fold_scores)

    f.write("OVERALL RESULTS (40 runs total):\n")
    f.write("-"*80 + "\n")
    f.write(f"Mean Macro F1: {np.mean(all_macro_f1):.4f} ± {np.std(all_macro_f1):.4f}\n")
    f.write(f"Min: {np.min(all_macro_f1):.4f}, Max: {np.max(all_macro_f1):.4f}\n\n")

    # Per-fold results
    for fold_num in [1, 2, 3, 4]:
        f.write(f"\n{'='*80}\n")
        f.write(f"FOLD {fold_num} RESULTS\n")
        f.write(f"{'='*80}\n\n")

        fold_results = all_fold_results[fold_num]
        macro_f1_scores = [r['macro_f1'] for r in fold_results]

        f.write(f"Macro F1 Scores (10 seeds):\n")
        f.write("-"*80 + "\n")
        for i, result in enumerate(fold_results):
            f.write(f"  Seed {result['seed']:5d}: {result['macro_f1']:.4f}\n")

        f.write(f"\nStatistics:\n")
        f.write(f"  Mean: {np.mean(macro_f1_scores):.4f} ± {np.std(macro_f1_scores):.4f}\n")
        f.write(f"  Min:  {np.min(macro_f1_scores):.4f}\n")
        f.write(f"  Max:  {np.max(macro_f1_scores):.4f}\n")

        # Per-class F1 (averaged across 10 seeds)
        f.write(f"\nPer-Class F1 Scores (averaged across 10 seeds):\n")
        f.write("-"*80 + "\n")

        # Collect all class names
        all_classes = set()
        for result in fold_results:
            all_classes.update(result['per_class_f1'].keys())

        # Average per-class F1 across seeds
        for class_name in sorted(all_classes):
            class_f1_scores = [r['per_class_f1'].get(class_name, 0) for r in fold_results]
            mean_f1 = np.mean(class_f1_scores)
            std_f1 = np.std(class_f1_scores)
            f.write(f"  {class_name:20s}: {mean_f1:.4f} ± {std_f1:.4f}\n")

print("✅ Results saved to 4fold_10seed_results.txt")

In [ ]:
# Display summary
print("\n" + "="*80)
print("SUMMARY - 4 FOLDS × 10 SEEDS = 40 TOTAL RUNS")
print("="*80 + "\n")

for fold_num in [1, 2, 3, 4]:
    macro_f1_scores = [r['macro_f1'] for r in all_fold_results[fold_num]]
    print(f"Fold {fold_num}: {np.mean(macro_f1_scores):.4f} ± {np.std(macro_f1_scores):.4f}")

all_macro_f1 = []
for fold_num in [1, 2, 3, 4]:
    all_macro_f1.extend([r['macro_f1'] for r in all_fold_results[fold_num]])

print(f"\n{'='*80}")
print(f"Overall Mean: {np.mean(all_macro_f1):.4f} ± {np.std(all_macro_f1):.4f}")
print(f"{'='*80}")